In [ ]:
# !pip install openai==0.28
# !pip install python-dotenv

In [ ]:
# additional information about OpenAI
AZURE_OPENAI_API_BASE='chatgpt-4-turbo'
AZURE_OPENAI_KEY='****************'
endpoint_name = "**********************"
deployment_id = AZURE_OPENAI_API_BASE
api_version = "2023-05-15"
api_key = "******************"

In [ ]:
# setup information about OpenAI
import os
import openai
from dotenv import load_dotenv
AZURE_OPENAI_API_BASE = os.getenv("AZURE_OPENAI_API_BASE")
openai.api_type = "azure"
openai.api_base = endpoint_name
openai.api_version = api_version
openai.api_key = api_key

In [ ]:
# for output
from IPython.display import display, Markdown

def chat(messages, max_tokens=1000):
    response = openai.ChatCompletion.create(
        engine=deployment_id,
        messages=messages,
        max_tokens=max_tokens
    )
    return response["choices"][0]["message"]["content"]

In [ ]:
import csv

category_list = list()
# path of csv file
input_file_path = "input.csv"   # colum = Category, Content
read_file_path = "read.csv"     # colum = Category, Content
output_file_path = "output.csv" # column = Content, ChatGPT, Answer, Check
prompt = """### 指示 ###
入力された文章を1つのCategoryに分類して"分類のみ"を出力してください．

"""
# prompt+="""\n### 入力例と出力例 ###\n形式は分類:文章です．\n\n"""

with open(input_file_path, mode='r', encoding='cp932') as file:
    reader = csv.DictReader(file)
    for i, row in enumerate(reader):
        # prompt+=f"{i+1}."+str(row['Category'])+"\n:"+str(row['Content'])+"\n\n"
        # prompt+=f"{i+1}."+str(row['Category'])+"\n:"+str(row['Content'])+"\n\n"
        category_list.append(str(row['Category']))

# remove duplicate categorizations
category_list = list(set(category_list))
# category_str = str(category_list).replace("[", "").replace("]", "")

prompt+="\n### 分類の種類 ###\n"
for c in category_list:
    prompt += c+", "

In [ ]:
# open csv file
output_answer_list = list()
output_content_list = list()
output_category_list = list()
output_check_list = list()
all_num = 0
correct_num = 0

# construct a new string by removing the numbers
def remove_digits(input_str):
    result = ''.join([char for char in input_str if not char.isdigit()])
    return result

with open(read_file_path, mode='r', encoding='cp932') as file:
    reader = csv.DictReader(file)

    for i, row in enumerate(reader):
        category = str(row['Category'])
        content = str(row['Content'])
        output_content=f"{i+1}."+str(content)+"\n"
        # print(f"Category\t:{category}\nContent\t:{content}")

        output_content_list.append(content)
        output_answer_list.append(category)
        # message = prompt+"\n### 文章 ###\n"+output_content[:100] + output_content[-100:]

        # if there are categories that are difficult to determine（classify categories with poor accuracy first）
        # prompt_assistant = """
        # ### 指示 ###
        # 入力された文章が「"※判断の困難なカテゴリ名"」に分類されるか考え，分類される場合True，分類されない場合False，を出力してください．
        # なお，「"※判断の困難なカテゴリ名"」の例文を下に示す

        # ### 例文 ###
        # 1. hogehoge 2. hogehoge 3. hogehoge 4. hogehoge 5. hogehoge
        # """

        message = prompt+"\n### 文章 ###\n"+output_content
        # GPT生成
        # 会話履歴を初期化
        conversation_history = [{"role": "system", "content": "You are a helpful assistant."}]
        conversation_history.append({"role": "user", "content": message})

        # レスポンス出力
        response = ""

        try:
            assistant_boolean = chat(conversation_history)
        except:
            assistant_boolean = "False"
            pass

        # if there are categories that are difficult to determine
        if assistant_boolean.lower() == "true":
            response = "※判断の困難なカテゴリ名"
        else:
            message = prompt+"\n### 文章 ###\n"+output_content
            # chat generation
            # initialize conversation history
            conversation_history = [{"role": "system", "content": "You are a helpful assistant."}]
            conversation_history.append({"role": "user", "content": message})
            response = ""
            try:
                response = chat(conversation_history)
            except:
                response = "Error"
                pass

            # response fluctuation elimination
            response_without_number = remove_digits(response)
            response_modified = response_without_number.replace(".", "").replace(" ", "")
            if ":" in response_modified:
                response_modified = response_modified.split(":")[0]
            output_category_list.append(response_modified)

        # Acuuracy calculation
        if response != "Error":
            check_flag = ""
            all_num+=1
            if category in str(response):
                correct_num+=1
                check_flag = "o"
            else:
                check_flag = "x"
            output_check_list.append(check_flag)
        print(f"{str(i).zfill(2)}.[{check_flag}]\tChatGPT: {response}\t正解:{category}")
        # display(Markdown(response_modified))
        # conversation_history.append({"role": "assistant", "content": response})

In [ ]:
# Acuuracy calculation (final result)
with open(output_file_path, mode='w', encoding='cp932', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, ['Content', 'ChatGPT', 'Answer', "Check"])
    writer.writeheader()
    for content, category, answer, check in zip(output_content_list, output_category_list, output_answer_list, output_check_list):
        writer.writerows([{'Content': content, 'ChatGPT': category, 'Answer': answer, "Check": check}])
print(f"accuracy => {float(correct_num)/all_num}")